# Linear 프로그램 튜터리얼(Linear Program (LP) Tutorial)
notebook 실행 방법은 [index](./index.ipynb)를 참조하자.


## Important Note
Drake에서 일반적인 최적화 프로그램 생성 및 풀기 위해서 [mathematical program tutorial](./mathematical_program.ipynb)를 참고하자.

## Linear Program
선형 프로그램(LP)는 특별한 유형의 최적화 문제이다. LP에서 비용과 제약 조건은 의사 결정 변수의 선형(친화(affine)) 함수이다. 일반적인 LP의 수학적 공식은 다음과 같다.

$\begin{aligned} \min_x \;c^Tx + d\\ \text{subject to } Ax\leq b \end{aligned}$

선형 계획 문제(LP)는 다양한 오픈소스 또는 상용 솔버를 통해 해결할 수 있다. Drake는 SCS, Gurobi, MOSEK™ 등 일부 솔버를 지원한다. 지원되는 솔버의 전체 목록은 [Doxygen page]( https://drake.mit.edu/doxygen_cxx/group__solvers.html)를 참조하자. Gurobi 및 MOSEK™과 같은 일부 상용 솔버는 사전 컴파일된 Drake 바이너리에 포함되지 않아 Deepnote/Colab/Binder에서 사용할 수 없다는 점에 유의하자.

Drake의 API는 선형 비용과 제약 조건을 추가하는 여러 함수를 지원한다. 이 자습서에서는 몇 가지 함수를 간략하게 살펴보자. 함수의 전체 목록은 [Doxygen](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html)을 확인하자.

### 선형 비용 추가하기(Add linear cost)
선형 비용을 추가하는 가장 쉬운 방법은 `AddLinearCost` 함수를 호출하는 것이다. 먼저 2개의 의사 결정 변수를 사용하여 최적화 프로그램을 구성하는 방법을 시연한 다음, 비용을 추가하기 위해 `AddLinearCost` 함수를 호출한다.

In [ ]:
from pydrake.solvers import MathematicalProgram, Solve
import numpy as np

# Create an empty MathematicalProgram named prog (with no decision variables,
# constraints or costs)
prog = MathematicalProgram()
# Add two decision variables x[0], x[1].
x = prog.NewContinuousVariables(2, "x")

새로운 선형 비용을 추가하기 위해 `AddLinearCost(expression)` 함수를 호출할 수 있다. 여기서 `expression`은 의사 결정 변수의 기호적 선형 표현식(symbolic linear expression)입니다.

In [ ]:
# Add a symbolic linear expression as the cost.
cost1 = prog.AddLinearCost(x[0] + 3 * x[1] + 2)
# Print the newly added cost
print(cost1)
# The newly added cost is stored in prog.linear_costs().
print(prog.linear_costs()[0])

만약 `AddLinearCost`를 다시 호출하면, `prog`에 저장된 총 비용은 모든 개별 비용들의 합이다. `prog.linear_costs()` 함수를 호출하면 두 개의 항목이 반환되는 것을 확인할 수 있다.

In [ ]:
cost2 = prog.AddLinearCost(2 * x[1] + 3)
print(f"number of linear cost objects: {len(prog.linear_costs())}")

선형 비용의 계수(coefficient)를 벡터로 알고 있다면, `AddLinearCost(e, f, x)` 함수를 호출하여 $e^Tx + f$ 형태의 선형 비용을 최적화 프로그램에 추가할 수도 있다.

In [ ]:
# We add a linear cost 3 * x[0] + 4 * x[1] + 5 to prog by specifying the coefficients
# [3., 4] and the constant 5 in AddLinearCost
cost3 = prog.AddLinearCost([3., 4.], 5., x)
print(cost3)

마지막으로, 사용자는 `AddCost`를 호출하여 선형 표현식을 선형 비용에 추가할 수 있다. Drake는 표현식의 구조를 분석하여 표현식이 선형이라고 판단되면 추가된 비용을 선형으로 간주한다.

In [ ]:
print(f"number of linear cost objects before calling AddCost: {len(prog.linear_costs())}")
# Call AddCost to add a linear expression as linear cost. After calling this function,
# len(prog.linear_costs()) will increase by 1.
cost4 = prog.AddCost(x[0] + 3 * x[1] + 5)
print(f"number of linear cost objects after calling AddCost: {len(prog.linear_costs())}")

### 선형 제약 추가하기(Add linear constraints)
3가지 유형의 선형 제약 조건이 있습니다.
  * 경계 상자 제약 조건(Bounding box constraint): 의사 결정 변수의 하한 및 상한을 지정 : $ lower \le x \le upper $
  * 선형 등식 제약 조건(Linear equality constraint): $Ax = b$
  * 선형 부등식 제약 조건(Linear inequality constraint): $lower <= Ax <= upper$
  
#### AddLinearConstraint and AddConstraint function
선형 제약 조건을 추가하는 가장 쉬운 방법은 3가지 유형의 모든 선형 제약 조건을 처리할 수 있는 `AddConstraint` 또는 `AddLinearConstraint` 함수를 호출하는 것이다. 일반적인 `AddConstraint` 함수와 비교할 때, `AddLinearConstraint`는 더 많은 제한 사항을 확인하고 선형이 아닌 제약 조건은 추가를 거부한다.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2, "x")
y = prog.NewContinuousVariables(3, "y")

# Call AddConstraint to add a bounding box constraint x[0] >= 1
bounding_box1 = prog.AddConstraint(x[0] >= 1)
print(f"number of bounding box constraint objects: {len(prog.bounding_box_constraints())}")

# Call AddLinearConstraint to add a bounding box constraint x[1] <= 2
bounding_box2 = prog.AddLinearConstraint(x[1] <= 2)
print(f"number of bounding box constraint objects: {len(prog.bounding_box_constraints())}")

# Call AddConstraint to add a linear equality constraint x[0] + y[1] == 3
linear_eq1 = prog.AddConstraint(x[0] + y[1] == 3.)
print(f"number of linear equality constraint objects: {len(prog.linear_equality_constraints())}")

# Call AddLinearConstraint to add a linear equality constraint x[1] + 2 * y[2] == 1
linear_eq2 = prog.AddLinearConstraint(x[1] + 2 * y[2] == 1)
print(f"number of linear equality constraint objects: {len(prog.linear_equality_constraints())}")

# Call AddConstraint to add a linear inequality constraint x[0] + 3*x[1] + 2*y[2] <= 4
linear_ineq1 = prog.AddConstraint(x[0] + 3*x[1] + 2*y[2] <= 4)
print(f"number of linear inequality constraint objects: {len(prog.linear_constraints())}")

# Call AddLinearConstraint to add a linear inequality constraint x[1] + 4 * y[1] >= 2
linear_ineq2 = prog.AddLinearConstraint(x[1] + 4 * y[1] >= 2)
print(f"number of linear inequality constraint objects: {len(prog.linear_constraints())}")

`AddLinearConstraint`는 제약 조건이 실제로 선형인지 확인하고, 선형이 아닌 경우 예외를 발생시킨다.

In [ ]:
# Add a nonlinear constraint square(x[0]) == 2 by calling AddLinearConstraint. This should
# throw an exception
try:
    prog.AddLinearConstraint(x[0] ** 2 == 2)
except RuntimeError as err:
    print(err.args)

제약 조건의 계수(coefficients)를 행렬로 알고 있는 경우, 사용자는 `AddLinearConstraint(A, lower, upper, x)` 함수를 호출하여 $lower \le Ax \le upper$ 제약 조건을 추가할 수도 있다. 이 메소드의 버전은 기호적 표현을 생성하지 않으며, 특히 `A`가 매우 클 때 더 효율적이다.

In [ ]:
# Add a linear constraint 2x[0] + 3x[1] <= 2, 1 <= 4x[1] + 5y[2] <= 3.
# This is equivalent to lower <= A * [x;y[2]] <= upper with
# lower = [-inf, 1], upper = [2, 3], A = [[2, 3, 0], [0, 4, 5]].
linear_constraint = prog.AddLinearConstraint(
    A=[[2., 3., 0], [0., 4., 5.]],
    lb=[-np.inf, 1],
    ub=[2., 3.],
    vars=np.hstack((x, y[2])))
print(linear_constraint)

#### AddBoundingBoxConstraint
경계 상자 제약 조건 ($lower \le x \le upper$)을 추가하는 경우 `AddConstraint` 또는 `AddLinearConstraint` 함수를 호출하는 것 외에도 `AddBoundingBoxConstraint(lower, upper, x)` 함수를 사용할 수 있다. 이 함수는 `AddConstraint` 및 `AddLinearConstraint`보다 약간 더 빠르다.

In [ ]:
# Add a bounding box constraint -1 <= x[0] <= 2, 3 <= x[1] <= 5
bounding_box3 = prog.AddBoundingBoxConstraint([-1, 3], [2, 5], x)
print(bounding_box3)

변수가 동일한 하한 또는 상한을 공유하는 경우 `AddBoundingBoxConstraint`에서 스칼라 `lower` 또는 `upper` 값을 사용할 수 있다. 예를 들어

In [ ]:
# Add a bounding box constraint 3 <= y[i] <= 5 for all i.
bounding_box4 = prog.AddBoundingBoxConstraint(3, 5, y)
print(bounding_box4)

#### AddLinearEqualityConstraint
선형 등식 제약 조건 ($ Ax = b$)을 추가하는 경우 `AddConstraint` 또는 `AddLinearConstraint` 함수를 호출하는 것 외에도 더 명확하고 (약간 더 빠르게) `AddLinearEqualityConstraint` 함수를 사용할 수 있다.

In [ ]:
# Add a linear equality constraint 4 * x[0] + 5 * x[1] == 1
linear_eq3 = prog.AddLinearEqualityConstraint(np.array([[4, 5]]), np.array([1]), x)
print(linear_eq3)

### Solving Linear Program.
일단 모든 제약 조건과 비용을 프로그램에 추가한 후에는 `Solve` 함수를 호출하여 프로그램을 풀고 `GetSolution`을 호출하여 결과를 얻을 수 있다.

In [ ]:
# Solve an optimization program
# min -3x[0] - x[1] - 5x[2] -x[3] + 2
# s.t 3x[0] + x[1] + 2x[2] = 30
#     2x[0] + x[1] + 3x[2] + x[3] >= 15
#     2x[1] + 3x[3] <= 25
#     -100 <= x[0] + 2x[2] <= 40
#   x[0], x[1], x[2], x[3] >= 0, x[1] <= 10
prog = MathematicalProgram()
# Declare x as decision variables.
x = prog.NewContinuousVariables(4)
# Add linear costs. To show that calling AddLinearCosts results in the sum of each individual
# cost, we add two costs -3x[0] - x[1] and -5x[2]-x[3]+2
prog.AddLinearCost(-3*x[0] -x[1])
prog.AddLinearCost(-5*x[2] - x[3] + 2)
# Add linear equality constraint 3x[0] + x[1] + 2x[2] == 30
prog.AddLinearConstraint(3*x[0] + x[1] + 2*x[2] == 30)
# Add Linear inequality constraints
prog.AddLinearConstraint(2*x[0] + x[1] + 3*x[2] + x[3] >= 15)
prog.AddLinearConstraint(2*x[1] + 3*x[3] <= 25)
# Add linear inequality constraint -100 <= x[0] + 2x[2] <= 40
prog.AddLinearConstraint(A=[[1., 2.]], lb=[-100], ub=[40], vars=[x[0], x[2]])
prog.AddBoundingBoxConstraint(0, np.inf, x)
prog.AddLinearConstraint(x[1] <= 10)

# Now solve the program.
result = Solve(prog)
print(f"Is solved successfully: {result.is_success()}")
print(f"x optimal value: {result.GetSolution(x)}")
print(f"optimal cost: {result.get_optimal_cost()}")